In [24]:
from models.AlexNet import AlexNet
from models.VGG13 import VGG13

from tensorflow.keras.datasets import cifar10
import tensorflow as tf
import numpy as np

### CIFAR-10

In [25]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [26]:
len(x_train), len(x_test)

(50000, 10000)

In [27]:
x_train = x_train / 255.0
x_test = x_test / 255.0

### AlexNet

In [31]:
alexnet = AlexNet((32, 32, 3), 10)
alexnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

alexnet.summary()

ValueError: Exception encountered when calling MaxPooling2D.call().

[1mComputed output size would be negative. Received: `inputs.shape=[None 0 0 256]` and `pool_size=[3 3]`.[0m

Arguments received by MaxPooling2D.call():
  • args=('<KerasTensor shape=(None, 0, 0, 256), dtype=float32, sparse=False, name=keras_tensor_160>',)
  • kwargs=<class 'inspect._empty'>

In [30]:
alexnet.fit(x_train, y_train, epochs=10)

Epoch 1/10


ValueError: Exception encountered when calling MaxPooling2D.call().

[1mNegative dimension size caused by subtracting 3 from 2 for '{{node AlexNet_1/max_pooling2d_38_1/MaxPool2d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 3, 3, 1], padding="VALID", strides=[1, 2, 2, 1]](AlexNet_1/conv2d_86_1/Relu)' with input shapes: [?,2,2,256].[0m

Arguments received by MaxPooling2D.call():
  • inputs=tf.Tensor(shape=(None, 2, 2, 256), dtype=float32)

### VGG13

In [21]:
vgg13 = VGG13((32,32, 3), 10)
vgg13.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

vgg13.summary()

/home/jao/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "VGG13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_67 (Conv2D)              │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_68 (Conv2D)              │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_69 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_70 (Conv2D)              │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_71 (Conv2D)              │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_72 (Conv2D)              │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_73 (Conv2D)              │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_74 (Conv2D)              │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_75 (Conv2D)              │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_76 (Conv2D)              │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_77 (Conv2D)              │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_78 (Conv2D)              │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_79 (Conv2D)              │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 4096)           │     2,101,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 10)             │        40,970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,638,218 (128.32 MB)

 Trainable params: 33,638,218 (128.32 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
vgg13.fit(x_train, y_train, epochs=10)

Epoch 1/10


I0000 00:00:1739483986.287426    2242 service.cc:148] XLA service 0x7f9638015550 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739483986.289945    2242 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2025-02-13 18:59:46.400458: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1739483986.704207    2242 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-02-13 18:59:47.436863: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1057', 196 bytes spill stores, 196 bytes spill loads

2025-02-13 18:59:47.513463: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1064', 104 by

   7/1563 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - accuracy: 0.1121 - loss: 2.8757

I0000 00:00:1739483992.736911    2242 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 562/1563 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - accuracy: 0.0977 - loss: 2.3388

KeyboardInterrupt: 